<a href="https://colab.research.google.com/github/AntNina/PyTorch_training/blob/main/Dataset_Dataloader_BatchNorm_Dropout_optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
from sklearn.datasets import fetch_california_housing

datasets = fetch_california_housing()

df = pd.DataFrame(datasets.data, columns=datasets.feature_names)

print(df.shape)

(20640, 8)


In [ ]:
import torch
import torch.nn as nn  
import torch.nn.functional as F

torch.manual_seed(2020)

x = torch.tensor(datasets.data, dtype=torch.float32)
t = torch.tensor(datasets.target, dtype=torch.float32)

tensorDataset = torch.utils.data.TensorDataset(x, t)

n_train = int(len(tensorDataset) * 0.7)
n_test = len(tensorDataset) - n_train 

train, test = torch.utils.data.random_split(tensorDataset, [n_train, n_test])

batch_size = 1024

train_dataloader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test, batch_size)

In [ ]:
class Net(nn.Module):

  def __init__(self, input_dim, hidden_dim, n_layers):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers

    self.input_layer = nn.Sequential(
        nn.BatchNorm1d(input_dim),
        nn.Linear(input_dim, hidden_dim),
        nn.ReLU(inplace=True)
    )

    middle = []
    for _ in range(n_layers):
      middle.append(nn.BatchNorm1d(hidden_dim),)
      middle.append(nn.Linear(hidden_dim, hidden_dim))
      middle.append(nn.ReLU(inplace=True))

    self.middle_layers = nn.Sequential(*middle)

    self.output_layer = nn.Linear(hidden_dim, 1)


  def forward(self, x):
    x = self.input_layer(x)
    x = self.middle_layers(x)
    x = self.output_layer(x)
    return x

In [ ]:
max_epoch = 30
criterion = nn.MSELoss()

def objective(trial):
  device = "cuda" if torch.cuda.is_available() else "cpu"

  n_layers = trial.suggest_int("n_layers", 1, 5)
  hidden_dim = trial.suggest_int("hidden_dim", 8, 128, 8)
  optimizer = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  learning_rate = trial.suggest_discrete_uniform("learning_rate", 1e-5, 1e-1, q=1e-5)

  net = Net(input_dim=8, hidden_dim=hidden_dim, n_layers=n_layers).to(device)
  optimizer = getattr(torch.optim, optimizer)(net.parameters(), lr=learning_rate)

  # train 
  net.train()
  for epoch in range(max_epoch):
    for batch in train_dataloader:
      x, t = batch
      x.to(device)
      t.to(device)

      optimizer.zero_grad()
      y = net(x)
      loss = criterion(y, t)
      loss.backward()
      optimizer.step()
    
  # validation
  net.eval()
  validation_loss = 0.0
  with torch.no_grad():
    for batch in test_dataloader:
      x, t = batch
      x.to(device)
      t.to(device)
      y = net(x)
      loss = criterion(y, t)
      validation_loss += loss.item() * x.size(0)

    validation_loss = validation_loss / len(test_dataloader.dataset)

  return validation_loss

In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 8.8 MB/s 
     |████████████████████████████████| 210 kB 47.9 MB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 146 kB 34.4 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 112 kB 40.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=e18b92dacb7c78631f1da0e3f6566a009e382ac00d508e7f0adfbf2cba8fb64e
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna

study = optuna.create_study()
study.optimize(objective, n_trials=50)

[I 2022-05-21 20:40:19,265] A new study created in memory with name: no-name-23291c9e-4764-4031-b365-d00fd388e6fe
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([111])) that is different to the input size (torch.Size([111, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([49])) that is different to the input size (torch.Size([49, 1])). This will li

In [ ]:

study.best_trial

'''
{'hidden_dim': 16,
 'learning_rate': 0.09012,
 'n_layers': 2,
 'optimizer': 'Adam'}
'''

study.trials_dataframe()


study.trials_dataframe().sort_values("value").head(10)

,number,value,datetime_start,datetime_complete,duration,params_hidden_dim,params_learning_rate,params_n_layers,params_optimizer,state
27,27,1.285379,2022-05-21 20:43:52.230493,2022-05-21 20:43:58.593455,0 days 00:00:06.362962,16,0.09012,2,Adam,COMPLETE
30,30,1.285412,2022-05-21 20:44:12.696610,2022-05-21 20:44:19.034616,0 days 00:00:06.338006,32,0.07893,2,Adam,COMPLETE
28,28,1.285466,2022-05-21 20:43:58.595469,2022-05-21 20:44:05.624794,0 days 00:00:07.029325,32,0.09137,3,Adam,COMPLETE
32,32,1.285648,2022-05-21 20:44:25.598006,2022-05-21 20:44:32.326261,0 days 00:00:06.728255,24,0.07877,3,Adam,COMPLETE
19,19,1.285893,2022-05-21 20:42:53.297857,2022-05-21 20:42:59.656016,0 days 00:00:06.358159,16,0.06760,2,Adam,COMPLETE
1,1,1.286310,2022-05-21 20:40:33.738334,2022-05-21 20:40:40.697621,0 days 00:00:06.959287,8,0.09732,4,Adam,COMPLETE
43,43,1.286474,2022-05-21 20:45:40.650006,2022-05-21 20:45:47.124220,0 days 00:00:06.474214,24,0.06347,2,Adam,COMPLETE
17,17,1.286497,2022-05-21 20:42:40.059209,2022-05-21 20:42:45.850375,0 days 00:00:05.791166,8,0.06698,1,Adam,COMPLETE
21,21,1.287044,2022-05-21 20:43:05.937881,2022-05-21 20:43:11.762042,0 days 00:00:05.824161,8,0.06500,1,Adam,COMPLETE
48,48,1.287128,2022-05-21 20:46:15.980454,2022-05-21 20:46:22.317856,0 days 00:00:06.337402,24,0.06758,2,Adam,COMPLETE
